In [4]:
# ruff: noqa: F401, E402
import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from binary_index_tree import BinIndexTree
from combinatorics import fib
from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView, splint

print()


In [150]:
from trees import Node, TreeNode

INF: int = 10**10


def min_bst_with_a_sum(r: Node, target: int) -> int:
    def dfs(n: Node):
        if not n:
            # Return INF min node count and generous interval.
            # mnc, nc, sum, mn, mx
            return INF, 0, 0, INF, -INF
        lmc, lc, ls, lmn, lmx = dfs(n.left)
        rmc, rc, rs, rmn, rmx = dfs(n.right)
        mc = min(lmc, rmc)
        nc = lc + rc + 1
        if mc < INF:
            # We found the min count under this node.
            # mn, nc, sum, mx, mnc
            return mc, 0, 0, 0, 0
        if not lmx < n.data < rmn:
            # This is not a BST.
            # Return INF min node count and prohibitive interval.
            # mnc, nc, sum, mn, mx
            return INF, 0, 0, -INF, INF
        s = ls + rs + n.data
        if s == target:
            # This node is a BST with fitting target sum.
            # mn, nc, sum, mx, mnc
            return nc, 0, 0, 0, 0
        # Propagate INFo upwards.
        # Handle interval brackets for missing children.
        return INF, nc, s, min(lmn, n.data), max(rmx, n.data)

    mc = dfs(r)[0]
    return int(mc) if mc < INF else -1
